In [ ]:
import pandas as pd
import numpy as np
import gdown
import os
import re

In [ ]:
!git clone https://github.com/TTHHA/DS105.git

Cloning into 'DS105'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 117 (delta 27), reused 32 (delta 13), pack-reused 60
Receiving objects: 100% (117/117), 840.96 KiB | 9.56 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [ ]:
def find_xlsx_files(directory):
    xlsx_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".xlsx"):
                xlsx_files.append(os.path.join(root, file))
    return xlsx_files

In [ ]:
def read_data():
  data_path ="/content/DS105/Hoà"
  data_path_list = find_xlsx_files(data_path)
  return data_path_list
list_path =read_data()

In [ ]:
a = [mai.replace('.xlsx',"") for mai in list_path]
b_temp = [mai.replace('/content/DS105/Hoà/',"") for mai in a]
b_temp = [item.replace("_","") for item in b_temp]
b_temp = [item.strip() for item in b_temp]
b_temp

In [45]:
def ffill_for_columns(df):
  check_pattern = re.compile(r'.*Unnamed.*')
  column_names = df.columns.tolist()
  none_list = []
  for i in column_names:
    if(check_pattern.match(str(i))):
      none_list.append(None)
    else:
      none_list.append(str(i))
  for i in range(1, len(none_list)):
    if pd.isna(none_list[i]):
      none_list[i] = none_list[i-1]

  df.columns = none_list
  return df
def get_sub_feature(df):
  list_unique = df.loc[0].unique().tolist()[1:]
  dict_index_unique = {}
  count = 0
  for i in list_unique:
    list_index_unique = [0]
    count = 0
    for j in df.loc[0]:
      if(j != i):
        count += 1
      else:
        list_index_unique.append(count)
        count += 1
    dict_index_unique[i] = list_index_unique
  return dict_index_unique

def transform_data(data_format, list_data_path, b_temp):
  result = pd.DataFrame()
  special_case = ["Tỷ suất nhập cư","Tỷ suất sinh thô, tỷ suất chết thô và tỷ lệ tăng tự nhiên của dân số phân theo địa phương chia theo Địa phương, Phân tổ và Năm"]
  df = pd.read_excel(list_data_path)
  df.columns = df.columns.astype(str)
  df.rename(columns={ df.columns[0]: "Province" }, inplace = True)
  check_pattern = re.compile(r'.*Unnamed.*')
  check_df = [x for x in df.columns if check_pattern.match(str(x))] # Chứa các cột tên Unnamed
  if (len(check_df) == 0): # Trường hợp không có sub feature # ok
    if(special_case[0] in list_data_path or special_case[1] in list_data_path):
      special_char_list_1 = ["Province"]
      special_char_list=[]
      list_rename = ["Province"]
      for i in df.columns:
        if ".1" in i:
          special_char_list_1.append(i)
          list_rename.append(i.replace('.1', ''))
        else:
          special_char_list.append(i)
      temp_data = df[special_char_list_1][1:]
      temp_data.columns = list_rename
      temp_data['Sub_Feature'] = df[special_char_list_1].iloc[0][1]

      temp_data_2 = df[special_char_list][1:]
      # temp_data_2.rename(columns={"Unnamed: 0": "Province" }, inplace = True)
      merged_df = pd.concat([temp_data,temp_data_2], ignore_index=True)
      merged_df['Main_Feature'] = b_temp
    else:
      df['Main_Feature'] = b_temp
      df['Sub_Feature'] = None
      df.columns = df.columns.astype(str)
      merged_df = pd.concat([data_format, df])
      if(len(merged_df.columns) != 26):
        print(merged_df.columns)

        print(list_data_path)
    result =  merged_df.copy()
  else: # Trường hợp có nhiều sub feature #ok
    merged_df = data_format.copy()
    df = ffill_for_columns(df)
    df.columns = df.columns.astype(str)
    sub_index = get_sub_feature(df) # dict về vị trí các sub_feature
    for i in sub_index:
      temp_df = df.iloc[1:, sub_index[i]]
      temp_df['Sub_Feature'] = i
      merged_df = pd.concat([merged_df,temp_df], ignore_index=True)
      # merged_df = merged_df.append(temp_df, ignore_index=True)
    merged_df['Main_Feature'] = b_temp
    if(len(merged_df.columns) != 26):
      print(merged_df.columns)
      print(list_data_path)
    result =  merged_df.copy()
  return result
def final_df(list_path,b_temp):
  final_df = pd.DataFrame([])
  count = 0
  for i in list_path:
    data_forrmat = pd.read_csv('/content/DS105/Data_final - Sheet1.csv')
    data_forrmat = data_forrmat.dropna()
    data_forrmat
    df = transform_data(data_forrmat,i,b_temp[count])
    count +=1
    final_df = pd.concat([final_df,df],ignore_index=True)
  return final_df

In [46]:
a = final_df(list_path,b_temp)
a

,Main_Feature,Province,Sub_Feature,2002,2003,2004,2005,2006,2007,2008,...,2015,2016,2017,2018,2019,2020,2021,Sơ bộ 2021,2022,Sơ bộ 2022
0,Tỷ suất chết của trẻ em dưới 1 tuổ...,CẢ NƯỚC,None,NaN,NaN,NaN,17.8,NaN,16.0,15,...,14.7,14.5,14.4,14.2,14,13.9,13.65,NaN,NaN,12.09
1,Tỷ suất chết của trẻ em dưới 1 tuổ...,Đồng bằng sông Hồng,None,NaN,NaN,NaN,11.5,NaN,10.0,11,...,11.7,11.5,11.3,11.2,11,10.96,10.25,NaN,NaN,9.27
2,Tỷ suất chết của trẻ em dưới 1 tuổ...,Hà Nội,None,NaN,NaN,NaN,7.9,NaN,7.5,7,...,10.2,10,9.8,9.8,9.6,9.1,9.22,NaN,NaN,8.48
3,Tỷ suất chết của trẻ em dưới 1 tuổ...,Hà Tây,None,NaN,NaN,NaN,20.6,NaN,13.0,..,...,..,..,..,..,..,..,..,NaN,NaN,..
4,Tỷ suất chết của trẻ em dưới 1 tuổ...,Vĩnh Phúc,None,NaN,NaN,NaN,15.0,NaN,14.8,12,...,12.9,12.7,12.5,12.4,12.2,12.95,12.1,NaN,NaN,11.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7950,"Số cơ sở khám, chữa bệnh trực thuo...",Cần Thơ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7951,"Số cơ sở khám, chữa bệnh trực thuo...",Hậu Giang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7952,"Số cơ sở khám, chữa bệnh trực thuo...",Sóc Trăng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7953,"Số cơ sở khám, chữa bệnh trực thuo...",Bạc Liêu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
a.Main_Feature.unique()

array(['Tỷ suất chết của trẻ em dưới 1 tuổi phân theo địa phương chia theo Địa phương và Năm',
       'Số nhân lực ngành Y trực thuộc sở Y tế phân theo địa phương',
       'Tỷ suất nhập cư, xuất cư và di cư thuần phân theo địa phương chia theo Tỉnh-Thành phố, Tỷ suất và Năm',
       'Năng suất lao động phân theo địa phương. Đơn vị tính Triệu đồngngười',
       'Chỉ số phát triển con người phân theo địa phương',
       'Diện tích, dân số và mật độ dân số phân theo địa phương chia theo Địa phương, Năm và Chỉ tiêu',
       'Tỷ lệ thiếu việc làm trong độ tuổi lao động phân theo địa phương. Đơn vị tính',
       'Tỷ suất sinh thô, tỷ suất chết thô và tỷ lệ tăng tự nhiên của dân số phân theo địa phương chia theo Địa phương, Phân tổ và Năm',
       'Số người nhiễm HIVAIDS phân theo địa phương chia theo Địa p

In [ ]:
a['Main_Feature'] = b

In [ ]:
a.to_csv("Ds105_612.csv")

In [ ]:
a = [mai.strip('.xlsx') for mai in main]
b = [mai.strip('/content/DS105/Hoà/') for mai in a]
b